Saleh Ahmad
i200605@nu.edu.pk

In [8]:
import numpy as np
from collections import defaultdict
import random

In [9]:
class Laplace_LM_Roman_Urdu():
    def __init__(self,ngram_type=None):
        if ngram_type == None:
            self.ngram = 2
        else:
            self.ngram = ngram_type

    def fit(self,TrainCorpusVocab): #TrainCorpus = all tokens in order of occurence
        Vocabs = defaultdict(int)
        UniqueVocabSize = defaultdict(int)
        VocabSize = defaultdict(int)
        for ngram in range(self.ngram,0,-1):
            IdxToken = 0
            ngrams = defaultdict(int)
            Len_Vocab = len(TrainCorpusVocab)
            while IdxToken < Len_Vocab-ngram+1:
                current_n_gram = TrainCorpusVocab[IdxToken]
                for idx in range(1,ngram):
                    current_n_gram += ' ' + TrainCorpusVocab[IdxToken+idx]
                
                ngrams[current_n_gram] += 1
                IdxToken += 1

            ngrams = {k:v for k,v in ngrams.items() if v > 0}
            Vocabs[ngram] = ngrams
            UniqueVocabSize[ngram] = len(ngrams)
            VocabSize[ngram] = sum(ngrams.values())
        
        self.Vocabs = Vocabs
        self.UniqueVocabSizes = UniqueVocabSize
        self.VocabSizes = VocabSize

    def __Get_Random_Length_Of_Verse(self):
        #generate random length b/w 5 and 9
        return random.randint(6,8)

    def __Get_Random_Starting_Token(self,s_tokens):
        #generate random starting token from s_tokens which is a list of starting tokens
        return random.choice(s_tokens)
    
    def __Print_Vocab(self):
        print(self.Vocabs)

    def __Get_Next_Word_From_Candidate(self,Sequence):
        ngram_ToUse = len(Sequence.split()) + 1
        Candidates = [ngram for ngram in self.Vocabs[ngram_ToUse].keys() if ' '.join(ngram.split()[0:-1]) == Sequence]
        if len(Candidates) == 0:
            return self.__Get_Next_Word_From_Candidate(' '.join(Sequence.split()[1:])) #BackOff
        else:
            Probs_Of_Candidates = np.zeros_like(Candidates)
            for idxCandidate,candidate in enumerate(Candidates):
                Probs_Of_Candidates[idxCandidate] = (self.Vocabs[ngram_ToUse][candidate] + 1) / ((self.VocabSizes[self.ngram] + self.UniqueVocabSizes[self.ngram]))
            return Candidates[np.argmax(Probs_Of_Candidates)].split()[-1] #return max prob and the next word

    def __ReverseBackOff(self,sequence): #Doesn't take into account the importance of any wordother than the first (Reverse Back Off Kinda?) (1....n-1 grams)
        '''Initially the length of the sequence will be one. It will try to complete the sequence to the self.ngrams-1 length mark.
        However all words added after the first one will be of equal importance. What it should do is that it should take into account
          the importance of the first word and then add the next word based on the importance of the first word and so on.
        '''
        ngram_To_Use = self.ngram - 1
        Candidates = [ngram for ngram in self.Vocabs[ngram_To_Use].keys() if ngram.split()[0] == sequence]
        Probs_Of_Candidates = np.zeros_like(Candidates)
        for idxCandidate,candidate in enumerate(Candidates):
            Probs_Of_Candidates[idxCandidate] = (self.Vocabs[ngram_To_Use][candidate] + 1) / ((self.VocabSizes[self.ngram] + self.UniqueVocabSizes[self.ngram]))
        return ' '.join(Candidates[np.argmax(Probs_Of_Candidates)].split()[1:]) #return max prob and the next word


    def predict(self,StartingTokens): #StartingTokens = list of repetitive tokens, can extract unique and count
        '''
        Prediction format
        1 stanza = 14 verses (1 * 14)
        1 verse = 6 - 8 words
        '''        

        Verse = ''
        for idxVerse in range(0,14):
            CurrentVerse = ''
            StartingTokenOfThisVerse = self.__Get_Random_Starting_Token(StartingTokens) #Verse starts with a random starting token, length is 1 right now
            VerseLength = self.__Get_Random_Length_Of_Verse()
            Sequence = StartingTokenOfThisVerse
            CurrentVerse += Sequence

            #Length is less than what is required by self.ngras
            if self.ngram > 2 and len(CurrentVerse.split()) == 1:
                NextWords = self.__ReverseBackOff(Sequence)
                Sequence += ' ' + NextWords
                CurrentVerse = Sequence

            #Now the sequence is ready for self.ngram probs
            for idxWord in range(len(CurrentVerse.split()),VerseLength):
                NextWords = self.__Get_Next_Word_From_Candidate(Sequence)
                Sequence = ' '.join(Sequence.split()[1:] + ([NextWords]))
                CurrentVerse += ' ' + NextWords
                
            CurrentVerse += '\n'
            if idxVerse != 0 and idxVerse % 2 == 1:
                CurrentVerse += '\n'
            Verse += CurrentVerse
        return Verse 

    def Print_Details(self):
        print('All ngrams',self.Vocabs.keys())
        for ngram in self.Vocabs.keys():
            print('ngram:',ngram)
            print('ngrams:',list(self.Vocabs[ngram].keys())[0:10])


In [10]:
ObjLM = Laplace_LM_Roman_Urdu(4)

AllTokens = np.load('../Data/All_Tokens_Roman_Urdu.npy')
AllTokens = [token.lower() for token in AllTokens]
StartingTokens = np.load('../Data/Starting_Tokens_Roman_Urdu.npy')
StartingTokens = [token.lower() for token in StartingTokens]
ObjLM.fit(AllTokens)

ObjLM.Print_Details()

All ngrams dict_keys([4, 3, 2, 1])
ngram: 4
ngrams: ['gulon mein rang bharay', 'mein rang bharay bad', 'rang bharay bad nobahar', 'bharay bad nobahar chalay', 'bad nobahar chalay chalay', 'nobahar chalay chalay bhi', 'chalay chalay bhi aao', 'chalay bhi aao ke', 'bhi aao ke gulshan', 'aao ke gulshan ka']
ngram: 3
ngrams: ['gulon mein rang', 'mein rang bharay', 'rang bharay bad', 'bharay bad nobahar', 'bad nobahar chalay', 'nobahar chalay chalay', 'chalay chalay bhi', 'chalay bhi aao', 'bhi aao ke', 'aao ke gulshan']
ngram: 2
ngrams: ['gulon mein', 'mein rang', 'rang bharay', 'bharay bad', 'bad nobahar', 'nobahar chalay', 'chalay chalay', 'chalay bhi', 'bhi aao', 'aao ke']
ngram: 1
ngrams: ['gulon', 'mein', 'rang', 'bharay', 'bad', 'nobahar', 'chalay', 'bhi', 'aao', 'ke']


In [11]:
Pred = ObjLM.predict(StartingTokens)

In [12]:
print(Pred)

mein le ke niklon ga –apne darmandah karwan
hum bhi gham e ishhq ko par ab

garmi kahan kuch khayaal aaya tha wehshat ka
go sab ko baham saghar o

meri na-ash ko kheinchain phiro ke mein
chaand se maand sitaron ne kaha aakhir shab

ik shore uthaya ghalib aah jo
daam شنیدن jis qader chahay bichaya mudda anqa

apni khamshi gungi goya har simt se
tamasha marey agay ik khail hai

sab ki nazrain bacha ke dekh
lartay hain aur haath mein talwar bhi

iqbal ka thikana abhi wohi kefiyat hai is
burhenapa payi wohi rahay gi magar naya khaar


